In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.sparse import csr_matrix, load_npz

import sys
import os
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    python_path = os.path.join(module_path, 'python')
    sys.path.append(python_path)
    
from dusa_function_lib import build_directory_city_name, get_ratings_df
from fairness_evaluation import GCE

In [4]:
TYPES_ORDER = {"minority":0, "nimby":1, "no_type":2}
MODEL_ORDER = {"rand":0, "pop": 1, "pop_nc":2,
               "ib": 3, "ub":4, "mf": 5,
               "cb_cat": 6, "cb_loc":7,
               "cbib_cat":8, "cbib_loc":9,
               "cbub_cat":10, "cbub_loc":11}


In [5]:
def create_dataframes(city, year, types):
    dir_city = build_directory_city_name(city)
    data_dir = f"../../data/rm/{dir_city}/{year}"
    item_cnv = pd.read_csv(f"{data_dir}/item_mapping.csv", sep="|")
    types_prop = pd.concat([types[types.itemId.isin(item_cnv.itemId)],
                            pd.DataFrame(data={"itemId":
                                        item_cnv[~item_cnv['itemId'].isin(types['itemId'])].itemId,
                                        'type': ['no_type']*item_cnv[~item_cnv['itemId']\
                                                                                .isin(types['itemId'])].shape[0],
                                               'ranking':[0]*item_cnv[~item_cnv['itemId']\
                                                                       .isin(types['itemId'])].shape[0]})
                            ]).sort_values('itemId').reset_index(drop=True)
    
    # delete duplicates
    types_prop = \
        types_prop.loc[types_prop.groupby('itemId').ranking.idxmin().reset_index().set_index('ranking').index]\
            .reset_index(drop=True)
    
    df_test_types_prop = get_ratings_df(load_npz(f'{data_dir}/num_comm_test.npz'),
                                        rm_info={'user_mapping':
                                                 pd.read_csv(f'{data_dir}/user_mapping.csv', sep='|'),
                                                 'item_mapping': item_cnv})\
    .merge(types_prop, how='left', on='itemId')
    
    df_train_types_prop = get_ratings_df(load_npz(f'{data_dir}/num_comm_train.npz'),
                                         rm_info={'user_mapping':
                                                  pd.read_csv(f'{data_dir}/user_mapping.csv', sep='|'),
                                                  'item_mapping': item_cnv})\
        .merge(types_prop, how='left', on='itemId')
    
    return item_cnv, types_prop, df_test_types_prop, df_train_types_prop

In [6]:
def fairness_results(city, year, item_mapping, types_prop, df_test_types_prop):
    dir_city = build_directory_city_name(city)
    data_dir = f"../../data/recommendations/{dir_city}/{year}/"
    model_list = os.listdir(f'{data_dir}')
    model_list = [f for f in model_list if f.endswith('.csv')]
    # model_list.remove('model_history.csv')

    gce_results = []
    pm_t = []
    for m in model_list:
        
        m_name = m.split('.')[0].split('rec_')[1]
        df_rec_m = pd.read_csv(f'{data_dir}{m}', sep='|')
        df_rec_m_types_prop = df_rec_m.merge(types_prop, how='left', on='itemId')

        types_prop_m = types_prop[(types_prop.itemId.isin(item_mapping.itemId)) & \
                              (types_prop.itemId.isin(df_rec_m.itemId))].reset_index(drop=True)

        types_prop_m_values= list(types_prop_m['type'].unique())
        types_prop_m_values.sort(key=lambda val: TYPES_ORDER[val])
        
        # Uniform
        p0 = dict(zip(types_prop_m_values, [1 / len(types_prop_m_values) for i in range(0, len(types_prop_m_values))]))
        # each = num_likes_test_cat/num_likes_test
        d =  df_test_types_prop.groupby('type').count()['itemId'].to_dict()
        d = [d[k] for k in sorted(TYPES_ORDER, key=TYPES_ORDER.get)]
        p1 = dict(zip(types_prop_m_values, [v/np.sum(d) for v in d]))
        # minority = 0.5, rest = 0.1
        p2 = dict(zip(types_prop_m_values, [0.8, 0.1, 0.1]))
        # nimby = 0.5, rest = 0.1
        p3 = dict(zip(types_prop_m_values, [0.1, 0.8, 0.1]))
        # no_type = 0.1, rest = 0.9/num_rest
        p7 = dict(zip(types_prop_m_values, [0.9 / (len(types_prop_m_values)-1) 
                                          for i in range(0, len(types_prop_m_values)-1)]+[0.1]))
        
        pf = {'p_uniform':p0, 'p_test':p1, 'p_minority':p2, 'p_nimby':p3, 'p_min_nimby':p7}

        gce_df = pd.DataFrame(index=[m_name])
        
        for n, p in zip(pf.keys(), pf.values()):

            gce_df[n], pm = GCE(itemIds=item_mapping.itemId.unique(),
                            df_rec_attributes=df_rec_m_types_prop, 
                            df_test_attributes=df_test_types_prop,
                            item_attributes=types_prop_m,
                            p_f=p,
                            fun='ndcg', beta=2, h=0.95, pc=0.0001)
        pm_t.append(pd.DataFrame(data=pm, index=[f'pm_{m_name}']))
        gce_results.append(gce_df)

    gce_results = pd.concat(gce_results)
    pm_t = pd.concat(pm_t)
    pm_t = pm_t[TYPES_ORDER.keys()]
    return gce_results.loc[[s for s in MODEL_ORDER.keys()]], pm_t.loc[[f'pm_{s}' for s in MODEL_ORDER.keys()]], pf

In [80]:
cities = ['Cambridge', 'Miami', 'New York']
years = ['2014', '2015', '2016', '2017']

In [77]:
types = dict()
for city in cities:
    types[city] = dict()
    city_directory_name = build_directory_city_name(city)
    for year in years: 
        types[city][year] = pd.read_excel(f'../../../Code/GeneratedExcels/{city_directory_name}_{year}.xlsx', sheet_name='cl_gpt', decimal=',')[['id', 'type', 'ranking']]
        types[city][year] = types[city][year].rename(columns={'id':'itemId'})
        types[city][year]['type'] = types[city][year]['type'].str.strip()

In [82]:
item_mapping = dict()
types_prop = dict()
df_test_types = dict()
df_train_types = dict()
for city in cities:
    item_mapping[city] = dict()
    types_prop[city] = dict()
    df_test_types[city] = dict()
    df_train_types[city] = dict()
    for year in years:
        item_mapping[city][year], types_prop[city][year], df_test_types[city][year], df_train_types[city][year] =\
            create_dataframes(city, year, types[city][year])

In [85]:
gce_results = dict()
pm = dict()
pf = dict()
for city in cities:
    gce_results[city] = dict()
    pm[city] = dict()
    pf[city] = dict()
    for year in years:
        gce_results[city][year], pm[city][year], pf[city][year] =\
            fairness_results(city, year, item_mapping[city][year], types_prop[city][year], df_test_types[city][year])

c:\Users\molin\Documents\TFG_Info\Recommendations\src\python\fairness_evaluation.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.06535629 0.06534074 0.0651993  ... 0.06566595 0.0654134  0.06521874]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rgis_df.loc[rgis_df.r > 0, 'ndcg'] = (2 ** rgis_df.loc[rgis_df.r > 0, 'rel_u_i'] - 1) / np.log2(
c:\Users\molin\Documents\TFG_Info\Recommendations\src\python\fairness_evaluation.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.06535629 0.06534074 0.0651993  ... 0.06566595 0.0654134  0.06521874]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  rgis_df.loc[rgis_df.r > 0, 'ndcg'] = (2 ** rgis_df.loc[rgis_df.r > 0, 'rel_u_i'] - 1) / np.log2(
c:\Users\molin\Documents\TFG_Info\Recommendation

In [89]:
for city in cities:
    for year in years:
        city_directory_name = build_directory_city_name(city)
        city_year_directory_name = f"../../data/final_metrics/{city_directory_name}/{year}"
        gce_results[city][year]['nDCG'] = pd.read_csv(f'{city_year_directory_name}/final_metrics.csv', index_col=0, sep='|')[['nDCG']]
        gce_results[city][year] = gce_results[city][year][['nDCG', 'p_uniform', 'p_test', 'p_minority', 'p_nimby', 'p_min_nimby']]

In [94]:
for city in cities:
    for year in years:
        city_directory_name = build_directory_city_name(city)
        city_year_directory_name = f"../../data/fairness/{city_directory_name}/{year}"
        gce_results_styled = gce_results[city][year].style.background_gradient().format('{:.5f}')
        print(f"City: {city}, Year: {year}")
        display(gce_results_styled)
        gce_results_styled.to_html(f"{city_year_directory_name}/final_fairness.html")

City: Cambridge, Year: 2014


,nDCG,p_uniform,p_test,p_minority,p_nimby,p_min_nimby
rand,0.04449,-0.19884,-0.02691,-0.32996,-2.07965,-0.56628
pop,0.30652,-0.19992,-0.01840,-0.29062,-2.09214,-0.55812
pop_nc,0.32462,-0.19687,-0.01586,-0.27974,-2.07315,-0.54888
ib,0.11413,-0.41551,-0.06066,-0.33049,-3.39046,-0.97456
ub,0.21818,-0.22213,-0.02715,-0.31766,-2.22668,-0.60827
mf,0.32875,-0.23851,-0.02118,-0.27924,-2.33182,-0.62923
cb_cat,0.04946,-0.14292,-0.01565,-0.30721,-1.72896,-0.45022
cb_loc,0.04322,-0.16032,-0.02686,-0.34915,-1.83270,-0.49534
cbib_cat,0.04243,-0.13671,-0.03032,-0.37307,-1.67437,-0.45345
cbib_loc,0.10131,-0.20676,-0.05948,-0.44998,-2.07803,-0.60274


City: Cambridge, Year: 2015


,nDCG,p_uniform,p_test,p_minority,p_nimby,p_min_nimby
rand,0.05851,-0.22129,-0.00579,-0.54184,-2.10887,-0.64074
pop,0.24466,-0.04800,-0.02604,-0.47183,-1.02663,-0.28260
pop_nc,0.24623,-0.10785,-0.01096,-0.88228,-1.11155,-0.43605
ib,0.11678,-0.20978,-0.00518,-0.53525,-2.04281,-0.61814
ub,0.17221,-0.22660,-0.00756,-0.51634,-2.15872,-0.64836
mf,0.23517,-0.09573,-0.00925,-0.81832,-1.08963,-0.40939
cb_cat,0.06901,-0.24876,-0.00761,-0.56256,-2.26172,-0.69474
cb_loc,0.04628,-0.34389,-0.01848,-0.62954,-2.78472,-0.87834
cbib_cat,0.03595,-0.11192,-0.01342,-0.43436,-1.48742,-0.41418
cbib_loc,0.06920,-0.07843,-0.00812,-0.69982,-1.07705,-0.36873


City: Cambridge, Year: 2016


...

,nDCG,p_uniform,p_test,p_minority,p_nimby,p_min_nimby
rand,0.04117,-0.10933,-0.01590,-0.75074,-1.24087,-0.43543
pop,0.35122,-0.13303,-0.02835,-0.48671,-1.59472,-0.46371
pop_nc,0.35199,-0.19761,-0.01540,-0.52647,-1.97386,-0.59396
ib,0.05584,-0.05611,-0.05627,-0.49603,-1.07262,-0.30433
ub,0.06843,-0.04471,-0.05244,-0.64536,-0.87927,-0.29048
mf,0.34822,-0.16145,-0.02053,-0.51054,-1.75964,-0.52237
cb_cat,0.01405,-0.10241,-0.01852,-0.70433,-1.23506,-0.41923
cb_loc,0.01901,-0.10349,-0.01896,-0.67334,-1.26795,-0.41985
cbib_cat,0.00997,-0.13977,-0.01037,-0.67656,-1.49986,-0.49297
cbib_loc,0.02962,-0.07869,-0.03449,-0.56836,-1.18309,-0.36097


City: New York, Year: 2016


,nDCG,p_uniform,p_test,p_minority,p_nimby,p_min_nimby
rand,0.03402,-0.06862,-0.04528,-0.56459,-1.11622,-0.33904
pop,0.27526,-0.03669,-0.00556,-0.28456,-0.94729,-0.20074
pop_nc,0.27434,-0.02040,-0.00246,-0.36768,-0.82476,-0.18812
ib,0.14508,-0.02069,-0.00125,-0.33830,-0.81870,-0.17731
ub,0.16515,-0.04826,-0.01204,-0.32478,-1.06688,-0.24998
mf,0.27296,-0.03044,-0.00776,-0.38698,-0.91826,-0.22294
cb_cat,0.01295,-0.06175,-0.03765,-0.53030,-1.09146,-0.32075
cb_loc,0.02512,-0.07014,-0.04760,-0.57747,-1.11746,-0.34340
cbib_cat,0.01082,-0.05437,-0.02652,-0.46264,-1.07863,-0.29591
cbib_loc,0.02415,-0.05515,-0.02341,-0.42613,-1.10143,-0.29174


City: New York, Year: 2017


,nDCG,p_uniform,p_test,p_minority,p_nimby,p_min_nimby
rand,0.01967,-0.10127,-0.00255,-0.43843,-1.41516,-0.39297
pop,0.32612,-0.05473,-0.04438,-0.23446,-1.05953,-0.22036
pop_nc,0.32720,-0.03334,-0.03656,-0.29659,-0.92433,-0.19728
ib,0.08715,-0.11553,-0.00293,-0.32303,-1.54793,-0.39878
ub,0.10492,-0.11940,-0.01072,-0.26414,-1.56973,-0.38758
mf,0.32699,-0.04813,-0.01594,-0.34317,-1.06726,-0.25570
cb_cat,0.01013,-0.12782,-0.00023,-0.36992,-1.61798,-0.43494
cb_loc,0.01074,-0.13297,-0.00092,-0.42488,-1.62848,-0.45512
cbib_cat,0.00743,-0.07341,-0.00586,-0.43975,-1.22568,-0.33451
cbib_loc,0.02153,-0.07940,-0.00687,-0.47819,-1.24671,-0.35289


In [95]:
for city in cities:
    for year in years:
        #city_directory_name = build_directory_city_name(city)
        #city_year_directory_name = f"../../data/fairness/{city_directory_name}/{year}"
        pm_results_styled = pm[city][year].style.background_gradient().format('{:.5f}')
        print(f"City: {city}, Year: {year}")
        display(pm_results_styled)
        # gce_results_styled.to_html(f"{city_year_directory_name}/final_pm.html")

City: Cambridge, Year: 2014


,minority,nimby,no_type
pm_rand,0.41066,0.12516,0.46418
pm_pop,0.43292,0.12455,0.44254
pm_pop_nc,0.43932,0.12548,0.43521
pm_ib,0.42107,0.08272,0.49621
pm_ub,0.41851,0.11835,0.46314
pm_mf,0.44193,0.11391,0.44416
pm_cb_cat,0.42037,0.14509,0.43455
pm_cb_loc,0.39888,0.13857,0.46256
pm_cbib_cat,0.38614,0.14879,0.46507
pm_cbib_loc,0.35538,0.12528,0.51934


City: Cambridge, Year: 2015


,minority,nimby,no_type
pm_rand,0.32244,0.12382,0.55374
pm_pop,0.34146,0.21323,0.44531
pm_pop_nc,0.23728,0.20233,0.56039
pm_ib,0.32428,0.12707,0.54864
pm_ub,0.33125,0.12147,0.54728
pm_mf,0.24907,0.20508,0.54585
pm_cb_cat,0.31653,0.11691,0.56656
pm_cb_loc,0.29898,0.09817,0.60285
pm_cbib_cat,0.35825,0.16301,0.47874
pm_cbib_loc,0.27445,0.20656,0.51899


City: Cambridge, Year: 2016


...

,minority,nimby,no_type
pm_rand,0.26340,0.18683,0.54977
pm_pop,0.33880,0.15458,0.50661
pm_pop_nc,0.32687,0.13065,0.54248
pm_ib,0.33299,0.20689,0.46013
pm_ub,0.28732,0.23680,0.47588
pm_mf,0.33123,0.14317,0.52560
pm_cb_cat,0.27389,0.18741,0.53871
pm_cb_loc,0.28150,0.18382,0.53468
pm_cbib_cat,0.28151,0.16218,0.55631
pm_cbib_loc,0.30996,0.19313,0.49691


City: New York, Year: 2016


,minority,nimby,no_type
pm_rand,0.31083,0.20127,0.48790
pm_pop,0.42784,0.22516,0.34700
pm_pop_nc,0.38375,0.24649,0.36975
pm_ib,0.39801,0.24767,0.35432
pm_ub,0.40622,0.20757,0.38621
pm_mf,0.37534,0.22984,0.39482
pm_cb_cat,0.32151,0.20443,0.47406
pm_cb_loc,0.30697,0.20113,0.49190
pm_cbib_cat,0.34511,0.20605,0.44884
pm_cbib_loc,0.35952,0.20303,0.43744


City: New York, Year: 2017


,minority,nimby,no_type
pm_rand,0.35621,0.16926,0.47453
pm_pop,0.46017,0.20866,0.33117
pm_pop_nc,0.42079,0.22887,0.35034
pm_ib,0.41035,0.15809,0.43156
pm_ub,0.44467,0.15641,0.39892
pm_mf,0.39680,0.20752,0.39568
pm_cb_cat,0.38726,0.15280,0.45994
pm_cb_loc,0.36294,0.15206,0.48500
pm_cbib_cat,0.35467,0.18816,0.45716
pm_cbib_loc,0.34016,0.18589,0.47395
